# Hands-on Session Chatbot
This is the hands-on session accompanying the workshop on LangChain fundamentals. This is inspired by the more extensive LangChain Cookbook Part 1.

Copyright (c) 2023 Michael Neumayr

## Setup

### 0. Set up the Colab in your drive

- Load this Colab from Github
- Run the first cell to install all required packages (this takes a moment)
- During installation jump to section "Set OpenAI API Key" and put the key we provide you instead of "PUT_YOUR_KEY_HERE"

### 1. Required python packages

In [ ]:
# install required packages; this may take some minutes; ignore dependency warnings it should work anyway
%pip install openai
%pip install langchain
%pip install pypdf
%pip install tiktoken

### 2. Load the workshop github

In [ ]:
!git clone https://github.com/michaelnoi/venture_labs_build.git

In [ ]:
%cd venture_labs_build
!git checkout only_static_files

### 3. OpenAI API key

In [1]:
import os

openai_api_key = os.getenv('OPENAI_API_KEY', 'PUT_YOUR_KEY_HERE')

## Project: Interactive Chatbot

### 0. Remember the list of messages

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage

chat = ChatOpenAI(openai_api_key=openai_api_key)

In [3]:
chat(
    [
        SystemMessage(content="Answer in Chinese."),
        HumanMessage(content="When is the Oktoberfest in Munich usually?"),
        AIMessage(content='The Oktoberfest in Munich usually begins in late September and lasts for 16-18 days, ending on the first Sunday in October or on October 3rd, German Unity Day, if it falls on a Monday.'),
        HumanMessage(content="And do you have recommendattions what to wear?"),
    ]
)

AIMessage(content='在慕尼黑的啤酒节上，人们通常穿着传统的巴伐利亚服装。男性可以穿着“lederhosen”（皮短裤）搭配一件方格衬衫、长袜和皮鞋。女性则可以选择“dirndl”（一种传统的连衣裙），通常配有围裙、襟饰和蕾丝。此外，还可以戴上一顶鸭舌帽或者传统的巴伐利亚帽子。这些传统服装可以在慕尼黑的专卖店或者在线商店购买到。如果你不打算穿传统服装，那么也可以穿着舒适的休闲装或者晚礼服参加啤酒节。')

Let's set up a proper interactive chatbot that stores the messages dynamically.

### 1. Conversation chain for a chatbot

<div class="alert" style="background-color: #151E35; color: #FFFFFF; border-color: #223358; border-width: 2px;">
    📎 <b>The ConversationChain includes</b>
    <ol>
        <li>a pre-defined prompt template for a conversation with the LLM. The template is filled with the user input and the chat history. See the template below. This saves you the prompt engineering part for a good conversation. If you like more flexibility, however, you can also use your own prompt template.</li>
        <li>per default a memory that stores the conversation history and append it to the prompt.</li>
    </ol>
</div>
</div>

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain

chat = ChatOpenAI(openai_api_key=openai_api_key)
chain = ConversationChain(llm=chat) #, verbose=True)

exit_conditions = ("quit", "exit")

The memory is empty at the beginning. The conversation chain will fill it automatically with the conversation history.

In [5]:
print(type(chain.memory.buffer))

<class 'str'>


Remember prompt templates. Here, the user input is automatically prefixed by the "Human: " keyword.

In [6]:
print(chain.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


Now let's set up a loop for a conversation with the LLM. The loop should ask the user for input, send the input to the LLM, and print the response. The loop should stop if the user enters "quit" or "exit".

In [7]:
while True:
    query = input("Human: ")
    if query in exit_conditions:
        print()
        print("AI: Goodbye!")
        break
    else:
        response = chain.predict(input=query)
        print()
        print(f"AI: {response}")
        print()


AI: Hello! I'm an AI, so I don't have emotions, but I'm here and ready to help you with any questions or topics you'd like to discuss. How can I assist you today?


AI: Sure! What genre or type of books are you interested in?


AI: If you're not sure, I can provide recommendations from a variety of genres. Some popular options include:

- "1984" by George Orwell: This classic dystopian novel explores themes of government surveillance and the dangers of totalitarianism.
- "To Kill a Mockingbird" by Harper Lee: This Pulitzer Prize-winning novel tackles issues of racial injustice and morality in the American South.
- "Pride and Prejudice" by Jane Austen: A beloved romance novel set in 19th-century England, it explores themes of societal expectations and personal growth.
- "The Great Gatsby" by F. Scott Fitzgerald: Set in the Jazz Age, this novel delves into themes of wealth, love, and the American Dream.
- "The Lord of the Rings" by J.R.R. Tolkien: A fantasy epic filled with adventure, f

Now check the memory again:

<div class="alert" style="background-color: #151E35; color: #A450E6">
    🎯 <b>TODO</b>
  <p>Now check the memory of the chain to see if it stored your conversation correctly. You can get a more readible format if you use buffer_as_messages.</p>
</div>

In [8]:
### TODO: print the stored memory of your conversation

print(chain.memory.buffer)

Human: Hey, how are you?
AI: Hello! I'm an AI, so I don't have emotions, but I'm here and ready to help you with any questions or topics you'd like to discuss. How can I assist you today?
Human: give me some book recommendations
AI: Sure! What genre or type of books are you interested in?
Human: 
AI: If you're not sure, I can provide recommendations from a variety of genres. Some popular options include:

- "1984" by George Orwell: This classic dystopian novel explores themes of government surveillance and the dangers of totalitarianism.
- "To Kill a Mockingbird" by Harper Lee: This Pulitzer Prize-winning novel tackles issues of racial injustice and morality in the American South.
- "Pride and Prejudice" by Jane Austen: A beloved romance novel set in 19th-century England, it explores themes of societal expectations and personal growth.
- "The Great Gatsby" by F. Scott Fitzgerald: Set in the Jazz Age, this novel delves into themes of wealth, love, and the American Dream.
- "The Lord of 

### 2. Longer conversations

<div class="alert" style="background-color: #151E35; color: #FFFFFF; border-color: #223358; border-width: 2px;">
    📎 <b>Hitting the token limit</b>
    <p>Like with handling large documents, you can also hit the token limit with conversations. But that is not just the case when your input is too large but also when the conversation is too long. This happens because you add the whole history to every prompt you make, so the history grows linearly with the number of interactions. This is bad for long conversations and there are multiple ways to fix this.</p>
    </p>
</div>

In [9]:
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationSummaryMemory, ConversationBufferWindowMemory

llm = OpenAI(openai_api_key=openai_api_key)
chat = ChatOpenAI(openai_api_key=openai_api_key)
chain = ConversationChain(llm=chat) #, verbose=True)

exit_conditions = ("quit", "exit")

#### i) Cut the conversation off after a specified limit

This is the simplest version. You can just cut off the conversation after a specified number of interactions. This won't run into any token limit problems but it will also not be able to handle long conversations.

<div class="alert" style="background-color: #151E35; color: #A450E6">
    🎯 <b>TODO</b>
  <p>The keyword <code style="color:#A450E6">k</code> is the window size, i.e. how many last interaction will be stored. Play around with k and tell the model something in a conversation and and ask it at some later point again.</p>
</div>

In [10]:
conversation = ConversationChain(
	llm=chat,
	memory=ConversationBufferWindowMemory(k=1)
)

In [11]:
while True:
    query = input("Human: ")
    if query in exit_conditions:
        print()
        print("AI: Goodbye!")
        break
    else:
        response = conversation.predict(input=query)
        print()
        print(f"AI: {response}")
        print()


AI: Yes, that's correct! Grass is typically green due to the presence of chlorophyll, which is responsible for absorbing sunlight and converting it into energy through the process of photosynthesis. The green color of grass is a result of chlorophyll's ability to absorb red and blue light while reflecting green light. This natural pigment is found in the chloroplasts of grass cells, giving it its vibrant green appearance.


AI: Actually, the color of water can vary depending on various factors such as the presence of impurities, the depth of the water, and the angle at which light hits it. In many cases, water appears to be blue due to selective absorption and scattering of light. Water molecules absorb longer-wavelength colors such as red and reflect shorter-wavelength colors like blue. This is why bodies of water, such as oceans and lakes, often appear blue or greenish-blue. However, it's important to note that clear water can also appear colorless or take on the color of its surrou

#### ii) Summarize the conversation and append the summary to the prompt

Another method is to summarize the conversation so far and append that instead of whole raw history to every prompt. This uses more tokens at the beginning but scales better for larger conversations.

<div class="alert" style="background-color: #151E35; color: #A450E6">
    🎯 <b>TODO</b>
  <p>Now utilize <code style="color:#A450E6">ConversationSummaryMemory</code> instead of the window memory. You can set it up in the same way, the only difference is, that the summary memory needs an llm (for the summary) as input instead of a window size. The keywork is <code style="color:#A450E6">llm=</code>.</p>
</div>

In [12]:
### TODO: set up the conversation chain with a ConversationSummaryMemory here

conversation = ConversationChain(
	llm=chat,
	memory=ConversationSummaryMemory(llm=chat)
)

Now let's try it out and print intermediate summaries

In [13]:
while True:
    query = input("Human: ")
    if query in exit_conditions:
        print()
        print("AI: Goodbye!")
        break
    else:
        response = conversation.predict(input=query)
        print()
        print(f"AI: {response}")
        print()


AI: I'm doing well, thank you! I'm an AI designed to assist with various tasks and provide information. How can I help you today?


AI: Sure! I can definitely help you with that. Here's a simple recipe for chicken curry:

Ingredients:
- 500g boneless chicken, cut into pieces
- 2 tablespoons vegetable oil
- 1 large onion, finely chopped
- 3 cloves of garlic, minced
- 1 tablespoon ginger, grated
- 2 teaspoons curry powder
- 1 teaspoon turmeric powder
- 1 teaspoon cumin powder
- 1 teaspoon coriander powder
- 1 teaspoon chili powder (adjust to taste)
- 1 cup coconut milk
- 1 cup chicken broth
- Salt, to taste
- Fresh cilantro, chopped (for garnish)
- Rice or naan bread (for serving)

Instructions:
1. Heat the vegetable oil in a large pan over medium heat. Add the chopped onion and sauté until it becomes translucent.
2. Add the minced garlic and grated ginger, and cook for another minute until fragrant.
3. Add the chicken pieces to the pan and cook until they are lightly browned on all sid

In [14]:
print(conversation.memory.buffer)

The human greets the AI and asks how it is doing. The AI responds that it is doing well and explains its purpose as an AI designed to assist with tasks and provide information. It asks the human how it can help. The human asks the AI for a recipe for chicken curry. The AI provides a detailed recipe for chicken curry, including a list of ingredients and step-by-step instructions. It also offers further assistance if needed.


#### iii) Compare token usage

There are also combinations of these techniques and more advanced methods, see below how the token usage scales with the number of interactions.

<img src="static/token_usage.png" width="1000"/>

Source: https://www.pinecone.io/learn/series/langchain/langchain-conversational-memory/

### 3. Streaming output, the real ChatGPT experience

<div class="alert" style="background-color: #151E35; color: #FFFFFF; border-color: #223358; border-width: 2px;">
    📎 <b>Streaming output</b>
    <p>Previously, you always had to wait for the full answer and only then was the result printed. Now, we want to have the real ChatGPT experience and also stream the output token by token as soon at is ready.</p>
</div>

We use the same conversation model, we just change how we call the chain. For streaming the outputs, we leverage the <code style="color:gray">streaming</code> keyword. This will return a generator that yields the tokens as they are generated. We can then print them one by one.

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

chat = ChatOpenAI(streaming=True, callbacks=[StreamingStdOutCallbackHandler()], openai_api_key=openai_api_key)
chain = ConversationChain(llm=chat)

In [16]:
answer = chat(
    [
        HumanMessage(content="Give me a poem about natural language processing"),
    ]
)

In the realm where words and data collide,
A poem of Natural Language Processing resides.
Where algorithms dance with linguistic flair,
Unraveling the mysteries of language, so rare.

In the vast expanse of textual verse,
NLP unlocks meaning, like a universe.
Sentences and phrases, in abundance they flow,
Through the digital realm, where insights bestow.

With every line, a token, carefully dissected,
Syntax and semantics, meticulously connected.
Parsing through structure, with computational might,
NLP unveils the beauty hidden in plain sight.

From sentiment analysis to sentiment classification,
NLP deciphers emotions, with precision and diction.
It understands the context, the subtlety, and tone,
Revealing the depths of what words can condone.

Topic modeling, it weaves a tapestry of themes,
Extracting knowledge from vast textual streams.
Clustering words and ideas, in a harmonious blend,
NLP brings coherence, where chaos may transcend.

Machine translation, across languages it roams

### 4. Chatbot with streaming output

<div class="alert" style="background-color: #151E35; color: #A450E6">
    🎯 <b>TODO</b>
  <p>Put together all the parts from above to create an interactive chatbot that has some memory and streams the output like you know from ChatGPT.</p>
</div>

In [17]:
### TODO: put your imports here

# use from above

In [18]:
### TODO: put your initializations here

chat = ChatOpenAI(streaming=True, callbacks=[StreamingStdOutCallbackHandler()], openai_api_key=openai_api_key)
conversation = ConversationChain(llm=chat)

In [ ]:
### TODO: put your loop here, you don't need to add different arguments to predict for streaming to work as it is already set up in the chat model

while True:
    query = input("Human: ")
    if query in exit_conditions:
        print()
        print("AI: Goodbye!")
        break
    else:
        response = conversation.predict(input=query)
        print()
        print(f"AI: {response}")
        print()

## More ressources

- Documentation: https://python.langchain.com/docs/get_started/introduction
- Really comprehensive tutorials: https://github.com/gkamradt/langchain-tutorials
- Deep dive conversational memory: https://www.pinecone.io/learn/series/langchain/langchain-conversational-memory/